In [21]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF

# Définir le chemin de sortie
output_dir = r"C:\Users\sbond\Desktop\spam_classifier_project\Resultat analyse et évaluation"
os.makedirs(output_dir, exist_ok=True)

# Charger le dataset
dataset = pd.read_csv(r"C:\\Users\\sbond\\Desktop\\spam_classifier_project\\SMSSpamCollection.csv", sep='\t', header=None, names=['Label', 'Message'])

# Diviser les données
X = dataset['Message']
y = dataset['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Vectoriser les textes
vectorizer = CountVectorizer(stop_words='english', max_features=5000)
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Entraîner le modèle
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_vect, y_train)

# Faire des prédictions
y_pred = classifier.predict(X_test_vect)
y_prob = classifier.predict_proba(X_test_vect)[:, 1]

# Générer le rapport de classification et la matrice de confusion
report = classification_report(y_test, y_pred, output_dict=True)
conf_matrix = confusion_matrix(y_test, y_pred)
roc_auc = round(roc_auc_score(y_test.map({'ham': 0, 'spam': 1}), y_prob), 2)
accuracy = round(accuracy_score(y_test, y_pred), 2)

# Créer un PDF pour inclure tous les visuels et tableaux structurés
class PDF(FPDF):
    def table_vertical(self, data, col_widths):
        self.set_font("Arial", size=8)
        header = data[0]
        rows = data[1:]

        table_width = sum(col_widths)
        x_start = (210 - table_width) / 2  # Centrer le tableau horizontalement

        self.set_x(x_start)
        # Ajouter les en-têtes de colonne
        for i, header_item in enumerate(header):
            self.cell(col_widths[i], 8, str(header_item), border=1, align='C', ln=0)
        self.ln(8)

        # Ajouter les lignes des données
        for row in rows:
            self.set_x(x_start)
            for i, item in enumerate(row):
                self.cell(col_widths[i], 8, str(item), border=1, align='C', ln=0)
            self.ln(8)

        # Si un tableau dépasse, ajuster dynamiquement les lignes
        if self.get_y() > 270:
            self.add_page()

pdf = PDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Section : Matrice de Confusion
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Non-Spam', 'Spam'], yticklabels=['Non-Spam', 'Spam'])
plt.title('Matrice de Confusion')
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.tight_layout()
conf_matrix_path = os.path.join(output_dir, 'matrice_confusion.png')
plt.savefig(conf_matrix_path)
pdf.add_page()
pdf.set_font("Arial", style='B', size=14)
pdf.cell(0, 10, txt="Matrice de Confusion", ln=True, align='C')
pdf.image(conf_matrix_path, x=10, y=30, w=190)
plt.close()

# Ajouter le tableau structuré des valeurs de la matrice de confusion
pdf.add_page()
pdf.set_font("Arial", style='B', size=14)
pdf.cell(0, 10, txt="Tableau des Valeurs - Matrice de Confusion", ln=True, align='C')
conf_data = [['', 'Non-Spam', 'Spam'], ['Non-Spam', *conf_matrix[0]], ['Spam', *conf_matrix[1]]]
pdf.table_vertical(conf_data, [40, 40, 40])

# Section : Performances globales
metrics = {"Précision": round(report['weighted avg']['precision'], 2),
           "Rappel": round(report['weighted avg']['recall'], 2),
           "F1-Score": round(report['weighted avg']['f1-score'], 2),
           "Exactitude": accuracy,
           "ROC AUC": roc_auc}
plt.figure(figsize=(10, 6))
bar = plt.bar(metrics.keys(), metrics.values(), color=['skyblue', 'orange', 'green', 'purple', 'red'])
for rect, value in zip(bar, metrics.values()):
    plt.text(rect.get_x() + rect.get_width() / 2, rect.get_height() / 2, f"{value:.2f}", ha='center', va='center', color='black')
plt.title('Métriques de Performance du Modèle')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.tight_layout()
performance_path = os.path.join(output_dir, 'performances.png')
plt.savefig(performance_path)
pdf.add_page()
pdf.set_font("Arial", style='B', size=14)
pdf.cell(0, 10, txt="Métriques de Performance", ln=True, align='C')
pdf.image(performance_path, x=10, y=30, w=190)
plt.close()

# Ajouter le tableau structuré des performances
pdf.add_page()
pdf.set_font("Arial", style='B', size=14)
pdf.cell(0, 10, txt="Tableau des Valeurs - Performances", ln=True, align='C')
pdf.table_vertical([['Métrique', 'Valeur']] + [[k, f"{v:.2f}"] for k, v in metrics.items()], [40, 40])

# Section : Répartition des Messages
labels = ['Non-Spam', 'Spam']
sizes = [sum(dataset['Label'] == 'ham'), sum(dataset['Label'] == 'spam')]
plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=['lightblue', 'salmon'])
plt.title('Répartition des Messages Non-Spam et Spam')
plt.axis('equal')
plt.tight_layout()
pie_chart_path = os.path.join(output_dir, 'repartition_messages.png')
plt.savefig(pie_chart_path)
pdf.add_page()
pdf.set_font("Arial", style='B', size=14)
pdf.cell(0, 10, txt="Répartition des Messages", ln=True, align='C')
pdf.image(pie_chart_path, x=10, y=30, w=190)
plt.close()

# Ajouter le tableau structuré des répartitions
pdf.add_page()
pdf.set_font("Arial", style='B', size=14)
pdf.cell(0, 10, txt="Tableau des Valeurs - Répartition", ln=True, align='C')
pdf.table_vertical([['Label', 'Count']] + [[l, c] for l, c in zip(labels, sizes)], [40, 40])

# Section : Longueur des Messages
dataset['Message_Length'] = dataset['Message'].apply(len)
plt.figure(figsize=(10, 6))
sns.histplot(data=dataset, x='Message_Length', hue='Label', kde=True, bins=30, palette='muted')
plt.title('Distribution des Longueurs de Messages par Étiquette')
plt.xlabel('Longueur du Message')
plt.ylabel('Fréquence')
plt.tight_layout()
histogram_path = os.path.join(output_dir, 'longueur_messages.png')
plt.savefig(histogram_path)
pdf.add_page()
pdf.set_font("Arial", style='B', size=14)
pdf.cell(0, 10, txt="Distribution des Longueurs de Messages", ln=True, align='C')
pdf.image(histogram_path, x=10, y=30, w=190)
plt.close()

# Ajouter le tableau structuré des statistiques des longueurs
pdf.add_page()
pdf.set_font("Arial", style='B', size=14)
pdf.cell(0, 10, txt="Tableau des Statistiques - Longueurs de Messages", ln=True, align='C')
length_stats = dataset.groupby('Label')['Message_Length'].describe().round(2)
pdf.table_vertical([['Statistique'] + length_stats.columns.tolist()] + [[idx] + row.tolist() for idx, row in length_stats.iterrows()], [20] * (len(length_stats.columns) + 1))

# Section : Nombre de Mots
dataset['Word_Count'] = dataset['Message'].apply(lambda x: len(x.split()))
plt.figure(figsize=(10, 6))
sns.violinplot(data=dataset, x='Label', y='Word_Count', palette='Set2')
plt.title('Distribution du Nombre de Mots par Étiquette')
plt.xlabel('Étiquette')
plt.ylabel('Nombre de Mots')
plt.tight_layout()
violinplot_path = os.path.join(output_dir, 'nombre_mots_violin.png')
plt.savefig(violinplot_path)
pdf.add_page()
pdf.set_font("Arial", style='B', size=14)
pdf.cell(0, 10, txt="Distribution du Nombre de Mots", ln=True, align='C')
pdf.image(violinplot_path, x=10, y=30, w=190)
plt.close()

# Ajouter le tableau structuré des statistiques des mots
pdf.add_page()
pdf.set_font("Arial", style='B', size=14)
pdf.cell(0, 10, txt="Tableau des Statistiques - Nombre de Mots", ln=True, align='C')
word_stats = dataset.groupby('Label')['Word_Count'].describe().round(2)
pdf.table_vertical([['Statistique'] + word_stats.columns.tolist()] + [[idx] + row.tolist() for idx, row in word_stats.iterrows()], [20] * (len(word_stats.columns) + 1))

# Exporter le rapport en PDF
pdf_path = os.path.join(output_dir, 'rapport_analyse.pdf')
pdf.output(pdf_path)

print("Fichiers générés:")
print(f"- PDF: {pdf_path}")
print(f"- Matrice de confusion: {conf_matrix_path}")
print(f"- Performances: {performance_path}")
print(f"- Répartition des messages: {pie_chart_path}")
print(f"- Histogramme des longueurs de messages: {histogram_path}")
print(f"- Distribution des mots (violon): {violinplot_path}")


C:\Users\sbond\AppData\Local\Temp\ipykernel_1312\142501453.py:170: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(data=dataset, x='Label', y='Word_Count', palette='Set2')


Fichiers générés:
- PDF: C:\Users\sbond\Desktop\spam_classifier_project\Resultat analyse et évaluation\rapport_analyse.pdf
- Matrice de confusion: C:\Users\sbond\Desktop\spam_classifier_project\Resultat analyse et évaluation\matrice_confusion.png
- Performances: C:\Users\sbond\Desktop\spam_classifier_project\Resultat analyse et évaluation\performances.png
- Répartition des messages: C:\Users\sbond\Desktop\spam_classifier_project\Resultat analyse et évaluation\repartition_messages.png
- Histogramme des longueurs de messages: C:\Users\sbond\Desktop\spam_classifier_project\Resultat analyse et évaluation\longueur_messages.png
- Distribution des mots (violon): C:\Users\sbond\Desktop\spam_classifier_project\Resultat analyse et évaluation\nombre_mots_violin.png
